# Full-Sequence Profile HMM for Methylation Detection

This notebook provides a comprehensive explanation of the **Full-Sequence Profile HMM** implemented in `methylation_hmm/full_sequence_hmm.py`. This is the primary model architecture for classifying cytosine modifications (C, 5mC, 5hmC) from Oxford Nanopore sequencing data.

---

## Table of Contents

1. [Model Overview](#section-1-overview)
2. [State Architecture](#section-2-states)
3. [Emission Distributions](#section-3-emissions)
4. [Transition Probabilities](#section-4-transitions)
5. [Classification Algorithm](#section-5-classification)
6. [Emission Parameter Sources](#section-6-params)
7. [Summary and Key Files](#section-7-summary)

---

<a id="section-1-overview"></a>
# Section 1: Model Overview

## 1.1 What is a Profile HMM?

A **Profile Hidden Markov Model (Profile HMM)** is a specialized HMM that models sequences of fixed length where each position has its own emission distribution. Unlike general HMMs where states can repeat, a profile HMM has:

- **One or more states per position** in the reference sequence
- **Position-specific emission parameters** trained from data
- **Sequential transitions** primarily moving forward through positions

## 1.2 Why Use a Full-Sequence Profile HMM?

Our model observes current at **every position** in the 155bp reference sequence, not just the 8 cytosine positions:

| Approach | Positions Observed | Context | Artifacts |
|----------|-------------------|---------|----------|
| **Sparse Model** | 8 cytosines only | None | Cannot model |
| **Full-Sequence HMM** | All 155 positions | Surrounding bases | Can model via transitions |

**Key advantage**: By modeling the entire sequence, we capture:
1. **5-mer context effects** - surrounding bases affect current levels
2. **Transition dynamics** - how the pore moves through DNA
3. **Sequential consistency** - a modification at position 38 should be consistent across the read

## 1.3 Model Architecture Summary

```
┌──────────────────────────────────────────────────────────────────────────────────────────────┐
│                        FULL-SEQUENCE PROFILE HMM ARCHITECTURE                                │
├──────────────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                              │
│  Reference:     A   T   G   C   A  ...   C   ...   T   G   C   ...   G   T   A              │
│  Position:      0   1   2   3   4  ...  38   ...  49  50  51  ...  153 154                  │
│                 │   │   │   │   │        │         │   │   │        │   │                   │
│                 ▼   ▼   ▼   ▼   ▼        ▼         ▼   ▼   ▼        ▼   ▼                   │
│  Observations: [o₀][o₁][o₂][o₃][o₄]... [o₃₈] ... [o₄₉][o₅₀][o₅₁]...[o₁₅₃][o₁₅₄]          │
│                                          ▲              ▲                                    │
│                                          │              │                                    │
│                                     FORK at 38     FORK at 50                                │
│                                     (cytosine)     (cytosine)                                │
│                                                                                              │
│  HMM States:   [M₀][M₁][M₂][M₃][M₄]...[Fork₃₈]...[M₄₉][Fork₅₀]...[M₁₅₃][M₁₅₄]             │
│                                           │              │                                   │
│                                      ┌────┼────┐    ┌────┼────┐                              │
│                                      │    │    │    │    │    │                              │
│                                     [C] [5mC][5hmC] [C] [5mC][5hmC]                          │
│                                                                                              │
│  Each observation oᵢ is a current measurement in picoamperes (pA)                           │
└──────────────────────────────────────────────────────────────────────────────────────────────┘
```

**Key insight**: At cytosine positions (38, 50, 62, 74, 86, 98, 110, 122), the HMM has multiple alternative states representing different modifications. The path taken through these "forks" determines the classification.

In [ ]:
# Imports used throughout this notebook
import json
import numpy as np
import pandas as pd
import torch
from pathlib import Path
from scipy.stats import norm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Project paths
PROJECT_ROOT = Path('.')
OUTPUT_DIR = PROJECT_ROOT / 'output' / 'rep1'
NANOPORE_DATA = PROJECT_ROOT / 'nanopore_ref_data'

# Model constants
SEQUENCE_LENGTH = 155
CYTOSINE_POSITIONS = [38, 50, 62, 74, 86, 98, 110, 122]
N_CYTOSINES = len(CYTOSINE_POSITIONS)  # 8
N_NON_CYTOSINES = SEQUENCE_LENGTH - N_CYTOSINES  # 147

print(f"Sequence length: {SEQUENCE_LENGTH} bp")
print(f"Cytosine positions: {CYTOSINE_POSITIONS}")
print(f"Number of cytosines: {N_CYTOSINES}")
print(f"Number of non-cytosines: {N_NON_CYTOSINES}")

---

<a id="section-2-states"></a>
# Section 2: State Architecture

## 2.1 State Types

The Full-Sequence HMM has **two types of states**:

| State Type | Where | Purpose | Count per Position |
|-----------|-------|---------|-------------------|
| **Match State** | Non-cytosine positions | Model expected current for that base | 1 |
| **Fork States** | Cytosine positions (8 total) | Model current for C, 5mC, or 5hmC | 2 (binary) or 3 (3-way) |

### Match States (Non-cytosine positions)

At positions that are **not** cytosines, there is exactly **one Match state** that represents the expected current distribution for that base in its 5-mer context.

```
Position 0 (base A):   [M₀] ← Single state with Gaussian emission
Position 1 (base T):   [M₁] ← Single state with Gaussian emission
...
Position 37 (base T):  [M₃₇] ← Single state with Gaussian emission
```

### Fork States (Cytosine positions)

At the 8 cytosine positions, there are **multiple alternative states** representing different modification states:

```
Position 38 (cytosine):
           ┌───────────────┐
           │    Fork 38    │
           ├───────────────┤
           │  [C]    ← Canonical cytosine state
           │  [5mC]  ← 5-methylcytosine state
           │  [5hmC] ← 5-hydroxymethylcytosine state (3-way mode only)
           └───────────────┘
```

**Binary mode**: 2 states per cytosine (C, 5mC)
**3-way mode**: 3 states per cytosine (C, 5mC, 5hmC)

## 2.2 Total State Count

The total number of states depends on the classification mode:

### Binary Mode (C vs 5mC)

```
Non-cytosine positions: 147 positions × 1 state = 147 states
Cytosine positions:       8 positions × 2 states = 16 states
─────────────────────────────────────────────────────────────
TOTAL:                                            163 states
```

### 3-Way Mode (C vs 5mC vs 5hmC)

```
Non-cytosine positions: 147 positions × 1 state = 147 states
Cytosine positions:       8 positions × 3 states = 24 states
─────────────────────────────────────────────────────────────
TOTAL:                                            171 states
```

### Formula

```python
n_states = (SEQUENCE_LENGTH - N_CYTOSINES) + (N_CYTOSINES × n_modifications)

# Binary:  (155 - 8) + (8 × 2) = 147 + 16 = 163
# 3-way:   (155 - 8) + (8 × 3) = 147 + 24 = 171
```

In [ ]:
# Calculate and verify state counts
def calculate_state_count(sequence_length, n_cytosines, n_modifications):
    """
    Calculate total number of HMM states.
    
    Args:
        sequence_length: Total positions in reference (155)
        n_cytosines: Number of cytosine positions (8)
        n_modifications: Number of modification states per cytosine (2 or 3)
    
    Returns:
        Total state count
    """
    n_non_cytosines = sequence_length - n_cytosines
    n_match_states = n_non_cytosines * 1  # 1 state per non-cytosine
    n_fork_states = n_cytosines * n_modifications  # n_modifications per cytosine
    return n_match_states + n_fork_states

print("=" * 70)
print("FULL-SEQUENCE HMM STATE COUNTS")
print("=" * 70)

print(f"\nReference sequence length: {SEQUENCE_LENGTH} bp")
print(f"Cytosine (fork) positions: {N_CYTOSINES} ({CYTOSINE_POSITIONS})")
print(f"Non-cytosine positions: {N_NON_CYTOSINES}")

# Binary mode
n_states_binary = calculate_state_count(SEQUENCE_LENGTH, N_CYTOSINES, 2)
print(f"\n--- Binary Mode (C vs 5mC) ---")
print(f"  Match states (non-cytosines): {N_NON_CYTOSINES} × 1 = {N_NON_CYTOSINES}")
print(f"  Fork states (cytosines):      {N_CYTOSINES} × 2 = {N_CYTOSINES * 2}")
print(f"  TOTAL STATES: {n_states_binary}")

# 3-way mode
n_states_3way = calculate_state_count(SEQUENCE_LENGTH, N_CYTOSINES, 3)
print(f"\n--- 3-Way Mode (C vs 5mC vs 5hmC) ---")
print(f"  Match states (non-cytosines): {N_NON_CYTOSINES} × 1 = {N_NON_CYTOSINES}")
print(f"  Fork states (cytosines):      {N_CYTOSINES} × 3 = {N_CYTOSINES * 3}")
print(f"  TOTAL STATES: {n_states_3way}")

print("=" * 70)

## 2.3 State Index Mapping

States are indexed sequentially by position. Here's how the mapping works:

```
Position 0 (non-cytosine):  State 0 = Match
Position 1 (non-cytosine):  State 1 = Match
...
Position 37 (non-cytosine): State 37 = Match

Position 38 (CYTOSINE):     State 38 = C fork
                            State 39 = 5mC fork
                            State 40 = 5hmC fork (3-way only)

Position 39 (non-cytosine): State 40 = Match (binary) or State 41 (3-way)
...
```

The implementation maintains a bidirectional mapping:
- `state_to_idx[(position, modification)]` → state index
- `idx_to_state[index]` → (position, modification)

In [ ]:
# Demonstrate state mapping
def build_state_mapping(sequence_length, cytosine_positions, modifications):
    """Build the state index mapping used by FullSequenceHMM."""
    state_to_idx = {}
    idx_to_state = {}
    
    idx = 0
    for pos in range(sequence_length):
        if pos in cytosine_positions:
            # Fork states for cytosines
            for mod in modifications:
                state_to_idx[(pos, mod)] = idx
                idx_to_state[idx] = (pos, mod)
                idx += 1
        else:
            # Single match state
            state_to_idx[(pos, 'match')] = idx
            idx_to_state[idx] = (pos, 'match')
            idx += 1
    
    return state_to_idx, idx_to_state, idx

# Build mapping for binary mode
mods_binary = ['C', '5mC']
state_to_idx, idx_to_state, n_states = build_state_mapping(
    SEQUENCE_LENGTH, set(CYTOSINE_POSITIONS), mods_binary
)

print("STATE INDEX MAPPING (Binary Mode)")
print("=" * 60)
print(f"\nTotal states: {n_states}")

# Show mapping around first cytosine (position 38)
print(f"\nMapping around position 38 (first cytosine):")
for pos in range(36, 42):
    if pos in CYTOSINE_POSITIONS:
        for mod in mods_binary:
            idx = state_to_idx[(pos, mod)]
            print(f"  Position {pos}, {mod:5s} → State {idx}")
    else:
        idx = state_to_idx[(pos, 'match')]
        print(f"  Position {pos}, match → State {idx}")

# Show final states
print(f"\nFinal states (positions 153-154):")
for pos in range(153, 155):
    if pos in CYTOSINE_POSITIONS:
        for mod in mods_binary:
            idx = state_to_idx[(pos, mod)]
            print(f"  Position {pos}, {mod:5s} → State {idx}")
    else:
        idx = state_to_idx[(pos, 'match')]
        print(f"  Position {pos}, match → State {idx}")

---

<a id="section-3-emissions"></a>
# Section 3: Emission Distributions

## 3.1 What is an Emission Distribution?

Each state has an **emission distribution** that defines the probability of observing a particular current value when in that state:

$$P(\text{observation} | \text{state}) \sim \mathcal{N}(\mu, \sigma^2)$$

- **Observation**: Mean current level in picoamperes (pA)
- **μ (mean)**: Expected current for that state
- **σ (std)**: Standard deviation capturing measurement noise

## 3.2 Emission Parameters Per State

**Every state has exactly ONE emission distribution** (univariate Gaussian):

| State Type | Position | # Emissions | Parameters |
|-----------|----------|-------------|------------|
| Match | Any non-cytosine | 1 | μ, σ from control data |
| C (fork) | Cytosine | 1 | μ, σ from control sample |
| 5mC (fork) | Cytosine | 1 | μ, σ from 5mC sample |
| 5hmC (fork) | Cytosine | 1 | μ, σ from 5hmC sample |

```
Match state at position 5:
┌─────────────────────────────────┐
│  State: M₅                      │
│  Emission: N(μ=812.3, σ=95.4)   │
└─────────────────────────────────┘

Fork states at position 38:
┌─────────────────────────────────┐
│  State: C₃₈                     │
│  Emission: N(μ=866.9, σ=106.2)  │
├─────────────────────────────────┤
│  State: 5mC₃₈                   │
│  Emission: N(μ=903.7, σ=108.5)  │
├─────────────────────────────────┤
│  State: 5hmC₃₈                  │
│  Emission: N(μ=875.8, σ=107.1)  │ (3-way mode only)
└─────────────────────────────────┘
```

In [ ]:
# Load emission parameters to show actual values
params_file = OUTPUT_DIR / 'hmm_3way_circuit_board.json'

if params_file.exists():
    with open(params_file) as f:
        params = json.load(f)
    
    print("=" * 80)
    print("EMISSION PARAMETERS AT CYTOSINE (FORK) POSITIONS")
    print("=" * 80)
    print("\nEach fork state has a single Gaussian emission distribution:")
    print("  P(observation | state) ~ Normal(μ, σ²)\n")
    
    print(f"{'Position':<10} {'State':<8} {'μ (pA)':<12} {'σ (pA)':<12} {'N samples':<10}")
    print("-" * 55)
    
    for dist in params['distributions'][:4]:  # Show first 4 positions
        pos = dist['position']
        for state in ['C', 'mC', 'hmC']:
            if state in dist:
                mean = dist[state]['mean']
                std = dist[state]['std']
                count = dist[state].get('count', 'N/A')
                state_name = 'C' if state == 'C' else ('5mC' if state == 'mC' else '5hmC')
                print(f"{pos:<10} {state_name:<8} {mean:<12.1f} {std:<12.1f} {count}")
        print()
    
    print("... (4 more cytosine positions)")
    print("\n" + "=" * 80)
    print("KEY INSIGHT: 5mC has ~+37 pA higher mean current than C")
    print("This difference in emission distributions enables classification")
    print("=" * 80)
else:
    print(f"Parameters file not found: {params_file}")

## 3.3 Emission Calculation

When observing current value $o$ at state with parameters $(\mu, \sigma)$:

$$P(o | \mu, \sigma) = \frac{1}{\sigma\sqrt{2\pi}} \exp\left(-\frac{(o - \mu)^2}{2\sigma^2}\right)$$

In practice, we use **log probabilities** to avoid numerical underflow:

$$\log P(o | \mu, \sigma) = -\frac{1}{2}\log(2\pi\sigma^2) - \frac{(o - \mu)^2}{2\sigma^2}$$

In [ ]:
# Demonstrate emission probability calculation
def log_emission_probability(observation, mean, std):
    """Calculate log P(observation | state) for Gaussian emission."""
    return norm.logpdf(observation, loc=mean, scale=std)

# Example: Position 38 with different observations
print("EMISSION PROBABILITY EXAMPLE")
print("=" * 70)
print("Position 38 (cytosine) emission parameters:")

# Typical parameters (from data)
c_params = {'mean': 866.9, 'std': 106.2}
mc_params = {'mean': 903.7, 'std': 108.5}
hmc_params = {'mean': 875.8, 'std': 107.1}

print(f"  C:    μ={c_params['mean']:.1f} pA, σ={c_params['std']:.1f} pA")
print(f"  5mC:  μ={mc_params['mean']:.1f} pA, σ={mc_params['std']:.1f} pA")
print(f"  5hmC: μ={hmc_params['mean']:.1f} pA, σ={hmc_params['std']:.1f} pA")

test_observations = [850, 880, 920]  # Low, medium, high current

print(f"\n{'Observation':<15} {'log P(o|C)':<15} {'log P(o|5mC)':<15} {'log P(o|5hmC)':<15} {'Most Likely':<12}")
print("-" * 72)

for obs in test_observations:
    log_p_c = log_emission_probability(obs, c_params['mean'], c_params['std'])
    log_p_mc = log_emission_probability(obs, mc_params['mean'], mc_params['std'])
    log_p_hmc = log_emission_probability(obs, hmc_params['mean'], hmc_params['std'])
    
    probs = {'C': log_p_c, '5mC': log_p_mc, '5hmC': log_p_hmc}
    most_likely = max(probs, key=probs.get)
    
    print(f"{obs:<15.1f} {log_p_c:<15.3f} {log_p_mc:<15.3f} {log_p_hmc:<15.3f} {most_likely:<12}")

print("\nLow current → C, High current → 5mC, Medium → could be C or 5hmC")

## 3.4 Visualizing Emission Distributions

The key to classification is that different modifications produce **different current distributions**:

In [ ]:
# Visualize emission distributions at position 38
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Position 38 distributions
x = np.linspace(600, 1100, 500)

c_params = {'mean': 866.9, 'std': 106.2}
mc_params = {'mean': 903.7, 'std': 108.5}
hmc_params = {'mean': 875.8, 'std': 107.1}

ax = axes[0]
ax.plot(x, norm.pdf(x, c_params['mean'], c_params['std']), 'b-', lw=2, label=f"C (μ={c_params['mean']:.0f})")
ax.plot(x, norm.pdf(x, mc_params['mean'], mc_params['std']), 'r-', lw=2, label=f"5mC (μ={mc_params['mean']:.0f})")
ax.plot(x, norm.pdf(x, hmc_params['mean'], hmc_params['std']), 'g-', lw=2, label=f"5hmC (μ={hmc_params['mean']:.0f})")

ax.axvline(c_params['mean'], color='b', linestyle='--', alpha=0.5)
ax.axvline(mc_params['mean'], color='r', linestyle='--', alpha=0.5)
ax.axvline(hmc_params['mean'], color='g', linestyle='--', alpha=0.5)

ax.set_xlabel('Current (pA)', fontsize=12)
ax.set_ylabel('Probability Density', fontsize=12)
ax.set_title('Emission Distributions at Position 38 (Cytosine)', fontsize=14)
ax.legend()
ax.grid(True, alpha=0.3)

# Annotation showing signal separation
ax2 = axes[1]
mods = ['C', '5mC', '5hmC']
means = [c_params['mean'], mc_params['mean'], hmc_params['mean']]
colors = ['blue', 'red', 'green']

bars = ax2.barh(mods, means, color=colors, alpha=0.7)
ax2.set_xlabel('Mean Current (pA)', fontsize=12)
ax2.set_title('Signal Separation Between Modifications', fontsize=14)

# Add delta annotations
ax2.annotate('', xy=(mc_params['mean'], 0.5), xytext=(c_params['mean'], 0.5),
            arrowprops=dict(arrowstyle='<->', color='black'))
ax2.text((c_params['mean'] + mc_params['mean'])/2, 0.65, f'Δ={mc_params["mean"]-c_params["mean"]:.0f} pA',
        ha='center', fontsize=10)

ax2.annotate('', xy=(hmc_params['mean'], 1.5), xytext=(c_params['mean'], 1.5),
            arrowprops=dict(arrowstyle='<->', color='black'))
ax2.text((c_params['mean'] + hmc_params['mean'])/2, 1.65, f'Δ={hmc_params["mean"]-c_params["mean"]:.0f} pA',
        ha='center', fontsize=10)

ax2.grid(True, alpha=0.3, axis='x')
ax2.set_xlim(800, 950)

plt.tight_layout()
plt.show()

print("KEY OBSERVATION:")
print(f"  5mC - C signal difference:  {mc_params['mean'] - c_params['mean']:.1f} pA (LARGE → good separation)")
print(f"  5hmC - C signal difference: {hmc_params['mean'] - c_params['mean']:.1f} pA (SMALL → harder to distinguish)")

---

<a id="section-4-transitions"></a>
# Section 4: Transition Probabilities

## 4.1 Transition Matrix Structure

The transition matrix $A$ defines $P(\text{next state} | \text{current state})$.

**Three types of transitions:**

| Transition Type | Probability | Meaning |
|----------------|-------------|--------|
| **Self-loop** | 0.10 | Dwelling on current base (oversegmentation) |
| **Forward** | 0.88 | Normal progression to next position |
| **Skip** | 0.02 | Skip next position (undersegmentation) |

```
From state at position i:

        ┌──────────────────────────────────────────────────┐
        │                                                  │
        │  P=0.10     P=0.88               P=0.02          │
        │  (self)     (forward)            (skip)          │
        │    │           │                   │             │
        │    ▼           ▼                   ▼             │
        │  ┌───┐     ┌───────┐          ┌───────┐          │
        └──│ i │────►│  i+1  │─────────►│  i+2  │          │
           └───┘     └───────┘          └───────┘          │
              ▲                                            │
              └────────────────────────────────────────────┘
```

## 4.2 Fork Transitions

When transitioning **into** a cytosine position, the probability is split equally among fork states:

```
Position 37 → Position 38 (cytosine):

                      P=0.88/2 = 0.44  ┌──────┐
                  ┌──────────────────►│  C   │
      ┌──────┐    │                   └──────┘
      │ M₃₇  │────┤
      └──────┘    │                   ┌──────┐
                  └──────────────────►│ 5mC  │
                      P=0.88/2 = 0.44 └──────┘
```

**Binary mode**: P = 0.88 / 2 = 0.44 per fork state
**3-way mode**: P = 0.88 / 3 ≈ 0.293 per fork state

In [ ]:
# Transition probability constants (from FullSequenceHMM)
P_SELF_LOOP = 0.10  # Stay at current position
P_FORWARD = 0.88    # Move to next position
P_SKIP = 0.02       # Skip to position + 2

print("TRANSITION PROBABILITIES")
print("=" * 60)
print(f"\nP(self-loop):  {P_SELF_LOOP:.2f}  → Dwelling (oversegmentation handling)")
print(f"P(forward):    {P_FORWARD:.2f}  → Normal progression")
print(f"P(skip):       {P_SKIP:.2f}  → Undersegmentation handling")
print(f"─" * 30)
print(f"Total:         {P_SELF_LOOP + P_FORWARD + P_SKIP:.2f}")

print("\n" + "=" * 60)
print("FORK TRANSITION PROBABILITIES")
print("=" * 60)

for n_mods, mode_name in [(2, 'Binary'), (3, '3-way')]:
    p_per_fork = P_FORWARD / n_mods
    print(f"\n{mode_name} mode ({n_mods} fork states):")
    print(f"  Forward to each fork state: {P_FORWARD:.2f} / {n_mods} = {p_per_fork:.3f}")
    print(f"  Skip to each fork state:    {P_SKIP:.2f} / {n_mods} = {P_SKIP/n_mods:.4f}")

## 4.3 Complete Transition Flow

Here's a visualization of transitions around a cytosine position:

In [ ]:
# Visualize transition structure around position 38
fig, ax = plt.subplots(figsize=(14, 6))

# Draw states
positions = [36, 37, 38, 39, 40]
y_positions = {'match': 0.5, 'C': 0.7, '5mC': 0.3}

# Draw match states
for pos in [36, 37, 39, 40]:
    circle = plt.Circle((pos, 0.5), 0.15, color='lightblue', ec='blue', lw=2)
    ax.add_patch(circle)
    ax.text(pos, 0.5, f'M{pos}', ha='center', va='center', fontsize=10, fontweight='bold')

# Draw fork states at position 38
for state, y in [('C', 0.7), ('5mC', 0.3)]:
    color = 'lightgreen' if state == 'C' else 'lightcoral'
    circle = plt.Circle((38, y), 0.15, color=color, ec='green' if state == 'C' else 'red', lw=2)
    ax.add_patch(circle)
    ax.text(38, y, state, ha='center', va='center', fontsize=10, fontweight='bold')

# Draw transitions
# From M37 to forks
ax.annotate('', xy=(38-0.15, 0.7), xytext=(37+0.15, 0.5),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))
ax.text(37.5, 0.65, 'P=0.44', fontsize=8)

ax.annotate('', xy=(38-0.15, 0.3), xytext=(37+0.15, 0.5),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))
ax.text(37.5, 0.35, 'P=0.44', fontsize=8)

# From forks to M39
ax.annotate('', xy=(39-0.15, 0.5), xytext=(38+0.15, 0.7),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))
ax.annotate('', xy=(39-0.15, 0.5), xytext=(38+0.15, 0.3),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

# Self-loops
for pos in [36, 37, 39, 40]:
    ax.annotate('', xy=(pos-0.1, 0.65), xytext=(pos+0.1, 0.65),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.8', color='blue', alpha=0.5))

# Regular forward transitions
for start, end in [(36, 37), (39, 40)]:
    ax.annotate('', xy=(end-0.15, 0.5), xytext=(start+0.15, 0.5),
                arrowprops=dict(arrowstyle='->', color='gray', lw=1.5))

# Position labels
for pos in positions:
    ax.text(pos, 0.1, f'pos {pos}', ha='center', fontsize=9)
    is_cytosine = pos == 38
    ax.text(pos, 0.05, 'Cytosine' if is_cytosine else 'Non-C', ha='center', fontsize=8, 
            color='red' if is_cytosine else 'gray')

ax.set_xlim(35.5, 40.5)
ax.set_ylim(0, 0.9)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Transition Structure Around Cytosine Position 38 (Binary Mode)', fontsize=14)

# Legend
legend_elements = [
    mpatches.Patch(color='lightblue', ec='blue', label='Match state'),
    mpatches.Patch(color='lightgreen', ec='green', label='C fork state'),
    mpatches.Patch(color='lightcoral', ec='red', label='5mC fork state'),
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

---

<a id="section-5-classification"></a>
# Section 5: Classification Algorithm

## 5.1 Classification Approach

The FullSequenceHMM classifies reads by computing the **total log-likelihood** of the observations assuming each modification type:

1. **For each modification (C, 5mC, 5hmC)**:
   - Compute emission probability at every position
   - At non-cytosine positions: use match state
   - At cytosine positions: use the fork state for this modification
   - Sum all log probabilities

2. **Classification**: Choose modification with highest total log-likelihood

```
score(C)    = Σᵢ log P(oᵢ | match_i) + Σⱼ log P(oⱼ | C_fork_j)
score(5mC)  = Σᵢ log P(oᵢ | match_i) + Σⱼ log P(oⱼ | 5mC_fork_j)
score(5hmC) = Σᵢ log P(oᵢ | match_i) + Σⱼ log P(oⱼ | 5hmC_fork_j)

prediction = argmax(score(C), score(5mC), score(5hmC))
```

**Note**: The difference between modification scores comes **only from the 8 cytosine positions** since non-cytosine positions have the same emission probability for all modifications.

In [ ]:
# Demonstrate classification algorithm
def classify_read(observations, emission_params, cytosine_positions):
    """
    Classify a read based on emission probabilities.
    
    Args:
        observations: Array of current values (one per position)
        emission_params: Dict mapping position -> {state -> (mean, std)}
        cytosine_positions: List of cytosine position indices
    
    Returns:
        Dict with scores, prediction, and probabilities
    """
    modifications = ['C', '5mC', '5hmC']
    scores = {mod: 0.0 for mod in modifications}
    
    for pos, obs in enumerate(observations):
        if pos not in emission_params:
            continue
            
        if pos in cytosine_positions:
            # Fork position: different score for each modification
            for mod in modifications:
                if mod in emission_params[pos]:
                    mean, std = emission_params[pos][mod]
                    scores[mod] += norm.logpdf(obs, mean, std)
        else:
            # Match position: same emission for all modifications
            mean, std = emission_params[pos]['match']
            log_p = norm.logpdf(obs, mean, std)
            for mod in modifications:
                scores[mod] += log_p
    
    # Convert to probabilities via softmax
    max_score = max(scores.values())
    exp_scores = {mod: np.exp(s - max_score) for mod, s in scores.items()}
    total = sum(exp_scores.values())
    probs = {mod: exp_scores[mod] / total for mod in modifications}
    
    prediction = max(scores, key=scores.get)
    
    return {
        'scores': scores,
        'probabilities': probs,
        'prediction': prediction,
        'confidence': probs[prediction] - sorted(probs.values())[-2]
    }

# Example with simplified parameters
print("CLASSIFICATION EXAMPLE")
print("=" * 70)

# Simplified emission params for demonstration
emission_params = {
    38: {'C': (866.9, 106.2), '5mC': (903.7, 108.5), '5hmC': (875.8, 107.1)},
    50: {'C': (845.2, 102.3), '5mC': (881.9, 104.1), '5hmC': (854.1, 103.5)},
}

# Test cases
test_cases = [
    ([850, 840], 'Low current at both cytosines → likely C'),
    ([900, 880], 'High current at both cytosines → likely 5mC'),
    ([870, 850], 'Medium current → ambiguous (likely C or 5hmC)'),
]

for obs_at_cytosines, description in test_cases:
    # Create observation array (only at cytosine positions)
    observations = {38: obs_at_cytosines[0], 50: obs_at_cytosines[1]}
    
    # Calculate scores manually
    scores = {'C': 0, '5mC': 0, '5hmC': 0}
    for pos, obs in observations.items():
        for mod in scores:
            mean, std = emission_params[pos][mod]
            scores[mod] += norm.logpdf(obs, mean, std)
    
    prediction = max(scores, key=scores.get)
    
    print(f"\n{description}")
    print(f"  Observations: pos38={obs_at_cytosines[0]} pA, pos50={obs_at_cytosines[1]} pA")
    print(f"  Log-scores: C={scores['C']:.2f}, 5mC={scores['5mC']:.2f}, 5hmC={scores['5hmC']:.2f}")
    print(f"  Prediction: {prediction}")

## 5.2 Confidence Scoring

The model outputs a **confidence score** based on the difference between the top two class probabilities:

$$\text{confidence} = P(\text{best}) - P(\text{second best})$$

**High confidence** (> 0.5): Clear separation between classes
**Low confidence** (< 0.2): Ambiguous classification

The evaluation framework uses confidence to compute **stratified accuracy**:
- Top 25% most confident: ~85% accuracy
- Top 50% most confident: ~83% accuracy  
- All reads: ~71% accuracy

---

<a id="section-6-params"></a>
# Section 6: Emission Parameter Sources

## 6.1 Single Adapter vs Pooled Parameters

A critical variable in the experiment is **where emission parameters come from**:

| Source | Description | Variance | Expected Accuracy |
|--------|-------------|----------|------------------|
| **Single adapter** | One 5-mer context (e.g., adapter_01) | Lower (~79 pA) | Higher |
| **Pooled** | All 32 adapters combined | Higher (~106 pA) | Lower |

**Why does this matter?**

The current produced by a nanopore depends on the **5-mer context** (the 5 bases around the pore). Pooling across all adapters mixes different 5-mer contexts, increasing variance.

```
Single adapter (adapter_01):
  - Same 5-mer context for all reads
  - Lower σ → tighter distribution
  - Better class separation (higher d' / Cohen's d)

Pooled (all adapters):
  - Mixed 5-mer contexts
  - Higher σ → wider distribution
  - More overlap between classes
```

In [ ]:
# Load and compare single vs pooled parameters
csv_file = OUTPUT_DIR / 'signal_at_cytosines_3way.csv'

if csv_file.exists():
    df = pd.read_csv(csv_file)
    
    # Single adapter stats
    df_single = df[df['chrom'] == '5mers_rand_ref_adapter_01']
    
    print("=" * 80)
    print("EMISSION PARAMETER COMPARISON: Single Adapter vs Pooled")
    print("=" * 80)
    
    pos = 38  # Example position
    
    print(f"\nPosition {pos} comparison:")
    print(f"{'Metric':<25} {'Single (adapter_01)':<20} {'Pooled (all)':<20} {'Difference':<15}")
    print("-" * 80)
    
    for sample, label in [('control', 'C'), ('5mC', '5mC')]:
        single_data = df_single[(df_single['sample'] == sample) & (df_single['position'] == pos)]['mean_current']
        pooled_data = df[(df['sample'] == sample) & (df['position'] == pos)]['mean_current']
        
        single_mean, single_std = single_data.mean(), single_data.std()
        pooled_mean, pooled_std = pooled_data.mean(), pooled_data.std()
        
        print(f"{label} mean (pA){'':<15} {single_mean:<20.1f} {pooled_mean:<20.1f} {single_mean - pooled_mean:+.1f}")
        print(f"{label} std (pA){'':<16} {single_std:<20.1f} {pooled_std:<20.1f} {single_std - pooled_std:+.1f}")
    
    # Calculate signal separation (d-prime / Cohen's d)
    c_single = df_single[(df_single['sample'] == 'control') & (df_single['position'] == pos)]['mean_current']
    mc_single = df_single[(df_single['sample'] == '5mC') & (df_single['position'] == pos)]['mean_current']
    c_pooled = df[(df['sample'] == 'control') & (df['position'] == pos)]['mean_current']
    mc_pooled = df[(df['sample'] == '5mC') & (df['position'] == pos)]['mean_current']
    
    d_single = (mc_single.mean() - c_single.mean()) / np.sqrt((c_single.std()**2 + mc_single.std()**2) / 2)
    d_pooled = (mc_pooled.mean() - c_pooled.mean()) / np.sqrt((c_pooled.std()**2 + mc_pooled.std()**2) / 2)
    
    print(f"\n{'Cohen\'s d (5mC vs C)':<25} {d_single:<20.3f} {d_pooled:<20.3f} {d_single - d_pooled:+.3f}")
    
    print("\n" + "=" * 80)
    print("KEY INSIGHT: Single-adapter has ~25% lower std and ~15% better separation (d')")
    print("This is why single-adapter parameters yield higher classification accuracy")
    print("=" * 80)
else:
    print(f"CSV file not found: {csv_file}")

## 6.2 The Four Evaluation Configurations

We evaluate 4 configurations to test the hypothesis that single-adapter parameters outperform pooled:

| # | Config Name | Mode | Emission Source | Expected Accuracy |
|---|-------------|------|-----------------|-------------------|
| 1 | `binary_single` | C vs 5mC | Single adapter | **Highest** |
| 2 | `binary_pooled` | C vs 5mC | All adapters | Medium |
| 3 | `3way_single` | C vs 5mC vs 5hmC | Single adapter | Medium |
| 4 | `3way_pooled` | C vs 5mC vs 5hmC | All adapters | Lowest |

**Hypothesis**: Single-adapter emission parameters have ~30% lower standard deviation, leading to better class separation and higher accuracy.

---

<a id="section-7-summary"></a>
# Section 7: Summary and Key Files

## 7.1 Model Summary Table

| Property | Binary Mode | 3-Way Mode |
|----------|-------------|------------|
| **Input** | ~155 current values (all positions) | ~155 current values |
| **Total States** | **163** | **171** |
| **Match States** | 147 (one per non-cytosine) | 147 |
| **Fork States** | 16 (2 per cytosine × 8) | 24 (3 per cytosine × 8) |
| **Emissions per State** | 1 Gaussian (μ, σ) | 1 Gaussian (μ, σ) |
| **Modifications** | C, 5mC | C, 5mC, 5hmC |
| **Classification** | Sum of log-likelihoods | Sum of log-likelihoods |

## 7.2 State Architecture Summary

```
┌────────────────────────────────────────────────────────────────────────────────────┐
│ FULL-SEQUENCE PROFILE HMM: STATE SUMMARY                                           │
├────────────────────────────────────────────────────────────────────────────────────┤
│                                                                                    │
│  Position Type        States per Position    Total (Binary)    Total (3-Way)      │
│  ─────────────────────────────────────────────────────────────────────────────    │
│  Non-cytosine (147)   1 Match state          147               147                │
│  Cytosine (8)         2 or 3 Fork states     16                24                 │
│  ─────────────────────────────────────────────────────────────────────────────    │
│  TOTAL                                       163 states        171 states         │
│                                                                                    │
│  Each state emits: P(current | state) ~ Normal(μ, σ²)                             │
│                                                                                    │
└────────────────────────────────────────────────────────────────────────────────────┘
```

## 7.3 Key Implementation Files

| File | Purpose |
|------|---------|
| **`methylation_hmm/full_sequence_hmm.py`** | `FullSequenceHMM` class - main classifier implementation |
| **`methylation_hmm/emission_params.py`** | Emission parameter computation (single vs pooled sources) |
| **`methylation_hmm/cli/run_evaluation.py`** | CLI for running single configuration evaluation |
| **`methylation_hmm/cli/run_all_evaluations.py`** | CLI for running all 4 configurations |
| **`output/rep1/signal_full_sequence.csv`** | Training data with current at all 155 positions |
| **`output/rep1/hmm_3way_circuit_board.json`** | Pre-computed emission parameters |

## 7.4 Running the Model

```bash
# Single configuration evaluation
python methylation_hmm/cli/run_evaluation.py \
    --mode binary \
    --emission-source single \
    --cv-folds 5 \
    --output results/full_evaluation

# Run all 4 configurations
python methylation_hmm/cli/run_all_evaluations.py \
    --output results/full_evaluation \
    --cv-folds 5
```

## 7.5 Key Results

| Configuration | Overall Accuracy | Top 25% Confidence | AUC |
|---------------|------------------|-------------------|-----|
| **binary_single** | **71.0%** | **85.0%** | 0.779 |
| binary_pooled | 63.3% | 76.6% | 0.684 |
| 3way_single | 52.0% | 54.5% | 0.674 |
| 3way_pooled | 46.9% | 48.5% | 0.615 |

**Key findings:**
1. Single-adapter beats pooled by **7.7 percentage points** in binary mode
2. Top 25% confidence reads achieve **85% accuracy** (confidence filtering works)
3. 3-way classification is harder due to 5hmC/C signal overlap (+9 pA vs +37 pA for 5mC)

---

# End of Notebook

This notebook has covered:

1. **Model Overview**: Full-sequence profile HMM observing all 155 positions
2. **State Architecture**: 163 states (binary) or 171 states (3-way)
   - Match states: 1 per non-cytosine position (147 total)
   - Fork states: 2-3 per cytosine position (16-24 total)
3. **Emission Distributions**: Each state has 1 Gaussian emission N(μ, σ²)
4. **Transition Probabilities**: Self-loop (0.10), forward (0.88), skip (0.02)
5. **Classification**: Sum of log-likelihoods across all positions
6. **Parameter Sources**: Single adapter (lower variance) vs pooled (higher variance)

For full implementation details, see `methylation_hmm/full_sequence_hmm.py`.